In [11]:
from astropy.time import Time
import numpy as np
import pandas as pd

import df_utils
from jupyterlabutils.notebook import EfdClient

In [12]:
#location = "tucson"
location = "summit"

In [13]:
client = EfdClient(location)

In [14]:
#csc = "ATMonochromator"
#csc = "Environment"
csc = "ATArchiver"
#csc = "ATMCS"
d1 = pd.Timestamp(2020, 1, 29).tz_localize('UTC')
d2 = pd.Timestamp(2020, 1, 31).tz_localize('UTC')
summary_state = 3  # FAULT
time_window = 10.0  # seconds
time_format = '%Y-%m-%dT%H:%M:%S.%f'

In [15]:
ss_df = await client.select_time_series(f"lsst.sal.{csc}.logevent_summaryState", 
                                        ["private_sndStamp",
                                         "private_rcvStamp",
                                         "summaryState"], d1, d2)

In [16]:
try:
    ss_df = df_utils.convert_timestamps(ss_df, ["private_sndStamp",
                                                "private_rcvStamp"])
except KeyError:
    print("NO DATA FOUND!!!!")

In [17]:
ec_df = await client.select_time_series(f"lsst.sal.{csc}.logevent_errorCode",
                                        ["private_sndStamp",
                                         "private_rcvStamp",
                                         "errorCode",
                                         "errorReport",
                                         "traceback"], d1, d2)
ec_df = df_utils.convert_timestamps(ec_df, ["private_sndStamp",
                                            "private_rcvStamp"])

In [18]:
timedelta = pd.Timedelta(time_window, unit='sec')

In [19]:
ss_df = ss_df[ss_df.summaryState == summary_state]

In [20]:
if ss_df.private_sndStamp_date.size:
    for timestamp in ss_df.private_sndStamp_date:
        print("######################################")
        print(f"Fault Summary State: {timestamp.strftime(time_format)}")
        t1 = timestamp - timedelta
        t2 = timestamp + timedelta
        error_indexes = ec_df.private_sndStamp_date.to_numpy()
        index = np.logical_and(error_indexes > t1, error_indexes < t2)
        error = ec_df[index]
        if error.shape[0] > 1:
            print("Multiple errorCode messages associated with Fault State!")
        for entry in range(error.shape[0]):
            print("----------------------------------")
            snd_stamp = error.private_sndStamp_date[entry].strftime(time_format)
            rcv_stamp = error.private_rcvStamp_date[entry].strftime(time_format)
            print(f"Fault State Report: {snd_stamp} ({rcv_stamp})")
            print(f"Error Code: {error.errorCode.values[entry]}")
            print(f"Error Report: {error.errorReport.values[entry]}")
            print(f"Traceback: {error.traceback.values[entry]}")
else:
    print(f"No fault states found for {csc}")

######################################
Fault Summary State: 2020-01-29T16:20:50.273252
----------------------------------
Fault State Report: 2020-01-29T16:20:50.270270 (2020-01-29T16:20:50.273722)
Error Code: 5751
Error Report: failed to received heartbeat ack from at_archive_controller
Traceback: 
######################################
Fault Summary State: 2020-01-29T17:35:16.457899
----------------------------------
Fault State Report: 2020-01-29T17:35:16.454982 (2020-01-29T17:35:16.457903)
Error Code: 5751
Error Report: failed to received heartbeat ack from at_archive_controller
Traceback: 
######################################
Fault Summary State: 2020-01-29T23:27:09.358141
----------------------------------
Fault State Report: 2020-01-29T23:27:09.355275 (2020-01-29T23:27:09.358161)
Error Code: 5755
Error Report: Forwarder is does not appear to be alive.  Going into fault state.
Traceback: 
######################################
Fault Summary State: 2020-01-30T07:21:11.142887
---